In [16]:
 #using mystats env
import pandas as pd
import numpy as np
import pingouin as pg
import association_metrics as am
import scipy.stats as stats
from scipy.stats import chisquare
from scipy import stats
import pyreadstat
import statsmodels.api as sm
import warnings
import os
import factor_analyzer
from factor_analyzer import FactorAnalyzer
pd.options.display.float_format = '{:.2f}'.format

In [17]:
#loading all the data
dfc=pd.read_csv('dfc.csv')
dfv=pd.read_csv('dfv.csv')
dfv_ColNames=pd.read_csv('dfv_ColNames.csv')
dfv_noDupe_ColNames=pd.read_csv('dfv_noDupe_ColNames.csv')
dfvnoDupe_recode=pd.read_csv('dfv_noDupe_recode.csv')
dfv_noDupe=pd.read_csv('dfv_noDupe.csv')
dfv_recode=pd.read_csv('dfv_recode.csv')
dfv_original=pd.read_csv('dfV_original.csv')
dfc_original=pd.read_csv('dfC_orignial.csv')

In [18]:
#add dfv variables into dfc
dfc.Case= dfc.Case.astype('category')
dfv.Case= dfv.Case.astype('category')
# merge dfc and selected columns from dfv_noDupe
dfc = pd.merge(dfc, dfv, on=['Case'], how='inner', indicator=True)

In [19]:
#testing Cronbach's alpha 1
trendiness = dfv_noDupe[['Humor', 'Animation', 'Filters', 'Music']]
pg.cronbach_alpha(data=trendiness)

(-0.11979958584937063, array([-0.527,  0.2  ]))

In [20]:
dfv_noDupe.columns

Index(['Case', 'Platform', 'Publisher', 'VideoURL', 'PubDate', 'Likes',
       'Comments', 'Saves', 'Shots', 'Graphics', 'EditsStyle', 'Broll',
       'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Offensive',
       'Sourcing', 'Fairness', 'Agenda', 'MediaIntegrity', 'Decorum',
       'Language', 'Topic', 'Presentation', 'ExternalVidSource', 'MediaType',
       'Animation', 'Filters', 'Decorum_Z', 'Language_Z'],
      dtype='object')

In [21]:
#testing Cronbach's alpha 2
trendinessAll = dfv_noDupe[['Shots', 'EditsStyle', 'Broll', 'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Sourcing', 'Decorum','Language','Animation', 'Filters', 'Fairness','Agenda']]
pg.cronbach_alpha(data=trendinessAll)

(0.32922840997276426, array([0.12 , 0.507]))

In [22]:
#testing Cronbach's alpha 3
trendiness = dfv_noDupe[['Music', 'Humor', 'Decorum','Language','Animation', 'Filters']]
pg.cronbach_alpha(data=trendiness)

(0.34129230687233253, array([0.119, 0.523]))

In [23]:
#testing Cronbach's alpha 4
trendiness = dfv_noDupe[['Music', 'Humor', 'Decorum','Language', 'Filters']]
pg.cronbach_alpha(data=trendiness)

(0.3545068558408392, array([0.13 , 0.535]))

In [24]:
#testing Cronbach's alpha 5
trendiness = dfv_noDupe[['EditsStyle','Music', 'Humor', 'Decorum','Language', 'Filters']]
pg.cronbach_alpha(data=trendiness)

(0.338014920748842, array([0.115, 0.52 ]))

In [25]:
#Running Factor Analysis since Cronbach's alpha is not high enough
fa=FactorAnalyzer(n_factors=6, rotation="varimax")#setting terms
fa.fit(trendinessAll)#applying terms for df with target variables
ev, v = fa.get_eigenvalues()#eigenvalues
EFAloaging=pd.DataFrame(fa.loadings_)#creating EFA loading df
EFAloaging['Variables'] = trendinessAll.columns#setting index
EFAloaging.to_csv('EFAloaging.csv')#saving EFA loading df
EFAloaging

,0,1,2,3,4,5,Variables
0,-0.01,0.15,0.09,0.87,0.04,0.08,Shots
1,0.51,-0.26,0.15,0.17,-0.01,0.12,EditsStyle
2,0.83,0.23,0.38,-0.33,0.11,-0.06,Broll
3,0.08,0.17,0.63,-0.16,0.15,0.05,NatSound
4,0.25,-0.65,0.03,-0.13,-0.06,-0.00,Audio
5,0.59,-0.20,-0.02,-0.00,-0.17,0.34,Music
6,0.65,-0.35,-0.09,-0.01,0.19,0.03,Narration
7,0.10,0.01,-0.48,0.06,-0.55,0.14,Humor
8,0.08,0.07,-0.00,0.06,0.52,0.08,Sourcing
9,0.09,0.60,0.24,0.06,-0.01,-0.14,Decorum


In [26]:
pd.DataFrame(ev)#factor variance

,0
0,2.94
1,2.51
2,1.58
3,1.38
4,1.14
5,0.93
6,0.76
7,0.68
8,0.63
9,0.61


In [27]:
#some modification so I can read the EFA table
trendinessAll2 = trendinessAll.copy()
trendinessAll2.columns = [str(i) + '_' + col for i, col in enumerate(trendinessAll.columns)]
trendinessAll2.columns

Index(['0_Shots', '1_EditsStyle', '2_Broll', '3_NatSound', '4_Audio',
       '5_Music', '6_Narration', '7_Humor', '8_Sourcing', '9_Decorum',
       '10_Language', '11_Animation', '12_Filters', '13_Fairness',
       '14_Agenda'],
      dtype='object')

In [28]:
trendinessAll.columns

Index(['Shots', 'EditsStyle', 'Broll', 'NatSound', 'Audio', 'Music',
       'Narration', 'Humor', 'Sourcing', 'Decorum', 'Language', 'Animation',
       'Filters', 'Fairness', 'Agenda'],
      dtype='object')

<h3>Cronbach's Alpha tested for the variables that were related in the factor analysis</h3>

In [29]:
#observed factors
factor0=['EditsStyle', 'Broll', 'Music', 'Narration']
factor1=['Audio', 'Decorum', 'Language']
factor2=['NatSound', 'Humor', 'Animation', 'Filters']
factor3=['Humor', 'Animation', 'Filters']
# factor4=['Broll', 'Music', 'Narration']
factor10=['EditsStyle', 'Broll', 'Music','Narration']
# factor11=['EditsStyle', 'Shots','Broll']

# factor12=['Humor', 'Decorum', 'Language']
#creating a factorlist
factorlist = [factor0, factor1, factor2, factor3, factor10]
for factor in factorlist:
    Cronbach = dfv_noDupe[factor]
    print(factor)
    print(pg.cronbach_alpha(data=Cronbach))

['EditsStyle', 'Broll', 'Music', 'Narration']
(0.7333301843717034, array([0.636, 0.809]))
['Audio', 'Decorum', 'Language']
(0.42581714273943805, array([0.199, 0.596]))
['NatSound', 'Humor', 'Animation', 'Filters']
(-0.6534402890641623, array([-1.254, -0.181]))
['Humor', 'Animation', 'Filters']
(-0.5765773809523809, array([-1.199, -0.108]))
['EditsStyle', 'Broll', 'Music', 'Narration']
(0.7333301843717034, array([0.636, 0.809]))


<h3>Since the following has satisfactory Cronbach's Alpha, we can use the variables in the factor analysis to create a composite score for TRENDINESS<br /> 
<br /> ['EditsStyle', 'Broll', 'Music', 'Narration']
<br /> (0.7133694189793809, array([0.622, 0.787]))


In [30]:
#adding 'trendiness' variable to dfv and dfv_noDupe 

# Define the factors
factor10=['EditsStyle', 'Broll', 'Music','Narration']

# Create new variables in dfv
dfv['trendiness'] = dfv[factor10].sum(axis=1)/4

# Create new variable, factor 10 in dfv_noDupe, and dfc
dfv_noDupe['trendiness'] = dfv_noDupe[factor10].sum(axis=1)/4
dfc['trendiness'] = dfc[factor10].sum(axis=1)/4


<h1>Now to Hypothesis Testing</h1>

<h2>H1a: Trendiness is more prevalent on TikTok compared to YouTube Shorts and Instagram Reels.</h2>

In [31]:
# Define the IV and DV variables
IV = 'Platform'
DV = ['trendiness']
DF = dfv
# Run the ANOVA
for i in DV:
    anova_result = pg.anova(data=DF, dv=i, between=IV)
    globals()[f"ANOVA_{i}"] = anova_result
    globals()[f"ANOVA_{i}"].insert(0, 'Variable', i)
    pairwise_comparisons = pg.pairwise_tests(data=DF, dv=i, between=IV, padjust='bonf')
    globals()[f"Pairwise_{i}"] = pairwise_comparisons
    globals()[f"Pairwise_{i}"].insert(0, 'Variable', i)
    mean_table = DF.groupby(IV)[i].mean().reset_index()
    std_table = DF.groupby(IV)[i].std().reset_index()
    mean_std_table = pd.merge(mean_table, std_table, on=IV)
    mean_std_table.columns = [i, 'Mean', 'Standard Deviation']
    mean_std_table = mean_std_table.rename(columns={i: 'Value'})
    globals()[f"mean_std_table_{i}"] = mean_std_table
    globals()[f"mean_std_table_{i}"].insert(0, 'Variable', i)
merged_anova = pd.concat([globals()[f"ANOVA_{i}"] for i in DV], axis=0)
merged_anova.reset_index(drop=True, inplace=True)
merged_pairwise = pd.concat([globals()[f"Pairwise_{i}"] for i in DV], axis=0)
merged_pairwise.reset_index(drop=True, inplace=True)
merged_mean_std_table = pd.concat([globals()[f"mean_std_table_{i}"] for i in DV], axis=0)
merged_mean_std_table.reset_index(drop=True, inplace=True)
# merged_anova.to_csv(IV+'/merged_anova.csv', index=False)
# merged_pairwise.to_csv(IV+'/merged_pairwise.csv', index=False)
# merged_mean_std_table.to_csv(IV+'/merged_mean_std.csv', index=False)
merged_anova

,Variable,Source,ddof1,ddof2,F,p-unc,np2
0,trendiness,Platform,2,123,0.06,0.95,0.00


In [32]:
merged_pairwise

,Variable,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,trendiness,Platform,Instagram Reels,TikTok,False,True,0.30,82.00,two-sided,0.77,1.00,bonf,0.237,0.06
1,trendiness,Platform,Instagram Reels,YouTube Shorts,False,True,0.00,82.00,two-sided,1.00,1.00,bonf,0.228,0.00
2,trendiness,Platform,TikTok,YouTube Shorts,False,True,-0.30,82.00,two-sided,0.77,1.00,bonf,0.236,-0.06


In [33]:
mean_std_table

,Variable,Value,Mean,Standard Deviation
0,trendiness,Instagram Reels,0.58,0.39
1,trendiness,TikTok,0.56,0.33
2,trendiness,YouTube Shorts,0.58,0.40


<h2>H1b: Social media only news outlets have content that is trendier compared to that of hybrid news outlets.
<br />H1c: Hybrid news outlets have content that is trendier compared to that of legacy news outlets.</h2>

In [34]:
# Define the IV and DV variables
IV = 'MediaType'
DV = ['trendiness']
DF = dfv_noDupe
# Run the ANOVA
for i in DV:
    anova_result = pg.anova(data=DF, dv=i, between=IV)
    globals()[f"ANOVA_{i}"] = anova_result
    globals()[f"ANOVA_{i}"].insert(0, 'Variable', i)
    pairwise_comparisons = pg.pairwise_tests(data=DF, dv=i, between=IV, padjust='bonf')
    globals()[f"Pairwise_{i}"] = pairwise_comparisons
    globals()[f"Pairwise_{i}"].insert(0, 'Variable', i)
    mean_table = DF.groupby(IV)[i].mean().reset_index()
    std_table = DF.groupby(IV)[i].std().reset_index()
    mean_std_table = pd.merge(mean_table, std_table, on=IV)
    mean_std_table.columns = [i, 'Mean', 'Standard Deviation']
    mean_std_table = mean_std_table.rename(columns={i: 'Value'})
    globals()[f"mean_std_table_{i}"] = mean_std_table
    globals()[f"mean_std_table_{i}"].insert(0, 'Variable', i)
merged_anova2 = pd.concat([globals()[f"ANOVA_{i}"] for i in DV], axis=0)
merged_anova2.reset_index(drop=True, inplace=True)
merged_pairwise = pd.concat([globals()[f"Pairwise_{i}"] for i in DV], axis=0)
merged_pairwise.reset_index(drop=True, inplace=True)
merged_mean_std_table = pd.concat([globals()[f"mean_std_table_{i}"] for i in DV], axis=0)
merged_mean_std_table.reset_index(drop=True, inplace=True)
# merged_anova.to_csv(IV+'/merged_anova.csv', index=False)
# merged_pairwise.to_csv(IV+'/merged_pairwise.csv', index=False)
# merged_mean_std_table.to_csv(IV+'/merged_mean_std.csv', index=False)
merged_anova

,Variable,Source,ddof1,ddof2,F,p-unc,np2
0,trendiness,Platform,2,123,0.06,0.95,0.00


In [35]:
merged_pairwise

,Variable,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,trendiness,MediaType,Hybrid,Legacy,False,True,-1.16,65.09,two-sided,0.25,0.75,bonf,0.445,-0.28
1,trendiness,MediaType,Hybrid,SMonly,False,True,-0.08,53.06,two-sided,0.94,1.00,bonf,0.247,-0.02
2,trendiness,MediaType,Legacy,SMonly,False,True,0.79,51.36,two-sided,0.43,1.00,bonf,0.338,0.20


In [36]:
merged_mean_std_table

,Variable,Value,Mean,Standard Deviation
0,trendiness,Hybrid,0.54,0.32
1,trendiness,Legacy,0.62,0.28
2,trendiness,SMonly,0.55,0.47


<h2>H2: Positive comment sentiment is higher on videos that have trendy presentations. (Comment sentiment is positively correlated to trendiness.)</h2>

In [37]:
pg.corr(dfc.trendiness, dfc.Sentiment, method='pearson').round(3)

,n,r,CI95%,p-val,BF10,power
pearson,979,-0.16,"[-0.22, -0.1]",0.00,8171.932,1.00


<h2>H3a: Videos with traditional presentations have less comments.</h2>

In [38]:
pg.corr(dfv.trendiness, dfv.Comments, method='pearson').round(3)

,n,r,CI95%,p-val,BF10,power
pearson,126,0.16,"[-0.02, 0.32]",0.08,0.511,0.42


<h2>H4a: Comments overall are a more negative sentiment compared to neutral or positive.

In [39]:
dfc.Sentiment.value_counts()

Sentiment
3.00    360
2.00    296
4.00    162
1.00    111
5.00     50
Name: count, dtype: int64

<h2>H4b: Comments on TikTok are more positive compared to Instagram Reels and YouTube Shorts.</h2>

<h2>Revised H4a: Comment sentiment is different depending on the platform.
<br />
H4b: Sentiment is higher on TikTok compared to Instagram Reels.
<br />
H4c: Sentiment is higher on TikTok compared to YouTube Shorts.
<br />
H4d: Sentiment is higher on Instagram Reels compared to YouTube Shorts.

In [40]:
# Define the IV and DV variables
IV = 'Platform_x'
DV = ['Sentiment']
DF = dfc
# Run the ANOVA
for i in DV:
    anova_result = pg.anova(data=DF, dv=i, between=IV)
    globals()[f"ANOVA_{i}"] = anova_result
    globals()[f"ANOVA_{i}"].insert(0, 'Variable', i)
    pairwise_comparisons = pg.pairwise_tests(data=DF, dv=i, between=IV, padjust='bonf')
    globals()[f"Pairwise_{i}"] = pairwise_comparisons
    globals()[f"Pairwise_{i}"].insert(0, 'Variable', i)
    mean_table = DF.groupby(IV)[i].mean().reset_index()
    std_table = DF.groupby(IV)[i].std().reset_index()
    mean_std_table = pd.merge(mean_table, std_table, on=IV)
    mean_std_table.columns = [i, 'Mean', 'Standard Deviation']
    mean_std_table = mean_std_table.rename(columns={i: 'Value'})
    globals()[f"mean_std_table_{i}"] = mean_std_table
    globals()[f"mean_std_table_{i}"].insert(0, 'Variable', i)
merged_anova = pd.concat([globals()[f"ANOVA_{i}"] for i in DV], axis=0)
merged_anova.reset_index(drop=True, inplace=True)
merged_pairwise = pd.concat([globals()[f"Pairwise_{i}"] for i in DV], axis=0)
merged_pairwise.reset_index(drop=True, inplace=True)
merged_mean_std_table = pd.concat([globals()[f"mean_std_table_{i}"] for i in DV], axis=0)
merged_mean_std_table.reset_index(drop=True, inplace=True)
# merged_anova.to_csv(IV+'/merged_anova.csv', index=False)
# merged_pairwise.to_csv(IV+'/merged_pairwise.csv', index=False)
# merged_mean_std_table.to_csv(IV+'/merged_mean_std.csv', index=False)
merged_anova

,Variable,Source,ddof1,ddof2,F,p-unc,np2
0,Sentiment,Platform_x,2,976,5.36,0.00,0.01


In [41]:
merged_pairwise

,Variable,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,Sentiment,Platform_x,Instagram Reels,TikTok,False,True,-3.07,761.73,two-sided,0.00,0.01,bonf,8.014,-0.22
1,Sentiment,Platform_x,Instagram Reels,YouTube Shorts,False,True,-2.21,512.18,two-sided,0.03,0.08,bonf,1.004,-0.18
2,Sentiment,Platform_x,TikTok,YouTube Shorts,False,True,0.60,476.80,two-sided,0.55,1.00,bonf,0.114,0.05


In [42]:
merged_mean_std_table

,Variable,Value,Mean,Standard Deviation
0,Sentiment,Instagram Reels,2.61,1.11
1,Sentiment,TikTok,2.84,0.99
2,Sentiment,YouTube Shorts,2.79,0.91


<h2>Report the following as descriptive statistics as the New RQ1:

<h3>Descriptive statistics per platform

In [43]:
# Define the IV and DV variables
IV = 'Platform'
DV = ['Shots', 'Animation', 'Filters', 'EditsStyle', 'Broll', 'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Sourcing']
DF = dfv
# Run the ANOVA
for i in DV:
    anova_result = pg.anova(data=DF, dv=i, between=IV)
    globals()[f"ANOVA_{i}"] = anova_result
    globals()[f"ANOVA_{i}"].insert(0, 'Variable', i)
    pairwise_comparisons = pg.pairwise_tests(data=DF, dv=i, between=IV, padjust='bonf')
    globals()[f"Pairwise_{i}"] = pairwise_comparisons
    globals()[f"Pairwise_{i}"].insert(0, 'Variable', i)
    mean_table = DF.groupby(IV)[i].mean().reset_index()
    std_table = DF.groupby(IV)[i].std().reset_index()
    mean_std_table = pd.merge(mean_table, std_table, on=IV)
    mean_std_table.columns = [i, 'Mean', 'Standard Deviation']
    mean_std_table = mean_std_table.rename(columns={i: 'Value'})
    globals()[f"mean_std_table_{i}"] = mean_std_table
    globals()[f"mean_std_table_{i}"].insert(0, 'Variable', i)
merged_anova = pd.concat([globals()[f"ANOVA_{i}"] for i in DV], axis=0)
merged_anova.reset_index(drop=True, inplace=True)
merged_pairwise = pd.concat([globals()[f"Pairwise_{i}"] for i in DV], axis=0)
merged_pairwise.reset_index(drop=True, inplace=True)
merged_mean_std_table = pd.concat([globals()[f"mean_std_table_{i}"] for i in DV], axis=0)
merged_mean_std_table.reset_index(drop=True, inplace=True)
# saving files
subfolder_path = IV+'/'
# Create the subfolder if it doesn't exist
if not os.path.exists(subfolder_path):
    os.makedirs(subfolder_path)
merged_anova.to_csv(IV+'/merged_anova.csv', index=False)
merged_pairwise.to_csv(IV+'/merged_pairwise.csv', index=False)
merged_mean_std_table.to_csv(IV+'/merged_mean_std.csv', index=False)

In [44]:
merged_anova

,Variable,Source,ddof1,ddof2,F,p-unc,np2
0,Shots,Platform,2,120,0.63,0.53,0.01
1,Animation,Platform,2,123,0.45,0.64,0.01
2,Filters,Platform,2,123,3.76,0.03,0.06
3,EditsStyle,Platform,2,123,0.48,0.62,0.01
4,Broll,Platform,2,123,0.40,0.67,0.01
5,NatSound,Platform,2,123,1.15,0.32,0.02
6,Audio,Platform,2,123,0.27,0.77,0.00
7,Music,Platform,2,123,0.79,0.46,0.01
8,Narration,Platform,2,123,0.00,1.00,0.00
9,Humor,Platform,2,123,0.26,0.77,0.00


In [45]:
merged_pairwise[merged_pairwise['Variable'] == 'Offensive']

,Variable,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges


In [46]:
merged_mean_std_table

,Variable,Value,Mean,Standard Deviation
0,Shots,Instagram Reels,0.72,0.46
1,Shots,TikTok,0.81,0.40
2,Shots,YouTube Shorts,0.71,0.46
3,Animation,Instagram Reels,0.40,0.50
4,Animation,TikTok,0.31,0.47
5,Animation,YouTube Shorts,0.33,0.48
6,Filters,Instagram Reels,0.02,0.15
7,Filters,TikTok,0.21,0.42
8,Filters,YouTube Shorts,0.12,0.33
9,EditsStyle,Instagram Reels,0.74,0.45


In [47]:
# Define the IV and DV variables
IV = 'Publisher'
DV = ['Shots', 'Animation', 'Filters', 'EditsStyle', 'Broll', 'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Sourcing', 'Decorum', 'Language']
DF = dfv_noDupe
# Run the ANOVA
for i in DV:
    anova_result = pg.anova(data=DF, dv=i, between=IV)
    globals()[f"ANOVA_{i}"] = anova_result
    globals()[f"ANOVA_{i}"].insert(0, 'Variable', i)
    pairwise_comparisons = pg.pairwise_tests(data=DF, dv=i, between=IV, padjust='bonf')
    globals()[f"Pairwise_{i}"] = pairwise_comparisons
    globals()[f"Pairwise_{i}"].insert(0, 'Variable', i)
    mean_table = DF.groupby(IV)[i].mean().reset_index()
    std_table = DF.groupby(IV)[i].std().reset_index()
    mean_std_table = pd.merge(mean_table, std_table, on=IV)
    mean_std_table.columns = [i, 'Mean', 'Standard Deviation']
    mean_std_table = mean_std_table.rename(columns={i: 'Value'})
    globals()[f"mean_std_table_{i}"] = mean_std_table
    globals()[f"mean_std_table_{i}"].insert(0, 'Variable', i)
merged_anova = pd.concat([globals()[f"ANOVA_{i}"] for i in DV], axis=0)
merged_anova.reset_index(drop=True, inplace=True)
merged_pairwise = pd.concat([globals()[f"Pairwise_{i}"] for i in DV], axis=0)
merged_pairwise.reset_index(drop=True, inplace=True)
merged_mean_std_table = pd.concat([globals()[f"mean_std_table_{i}"] for i in DV], axis=0)
merged_mean_std_table.reset_index(drop=True, inplace=True)
# saving files
subfolder_path = IV+'/'
# Create the subfolder if it doesn't exist
if not os.path.exists(subfolder_path):
    os.makedirs(subfolder_path)
merged_anova.to_csv(IV+'/merged_anova.csv', index=False)
merged_pairwise.to_csv(IV+'/merged_pairwise.csv', index=False)
merged_mean_std_table.to_csv(IV+'/merged_mean_std.csv', index=False)

/Users/stephenwsong/anaconda3/envs/mystats/lib/python3.11/site-packages/scipy/stats/_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/Users/stephenwsong/anaconda3/envs/mystats/lib/python3.11/site-packages/scipy/stats/_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/Users/stephenwsong/anaconda3/envs/mystats/lib/python3.11/site-packages/scipy/stats/_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/Users/stephenwsong/anaconda3/e

/Users/stephenwsong/anaconda3/envs/mystats/lib/python3.11/site-packages/scipy/stats/_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/Users/stephenwsong/anaconda3/envs/mystats/lib/python3.11/site-packages/pingouin/effsize.py:804: RuntimeWarning: invalid value encountered in scalar divide
  d = (x.mean() - y.mean()) / poolsd
/Users/stephenwsong/anaconda3/envs/mystats/lib/python3.11/site-packages/pingouin/effsize.py:804: RuntimeWarning: invalid value encountered in scalar divide
  d = (x.mean() - y.mean()) / poolsd
/Users/stephenwsong/anaconda3/envs/mystats/lib/python3.11/site-packages/scipy/stats/_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hyp

In [48]:
merged_anova

,Variable,Source,ddof1,ddof2,F,p-unc,np2
0,Shots,Publisher,5,94,1.11,0.36,0.06
1,Animation,Publisher,5,94,15.69,0.00,0.45
2,Filters,Publisher,5,94,9.89,0.00,0.34
3,EditsStyle,Publisher,5,94,8.96,0.00,0.32
4,Broll,Publisher,5,94,13.79,0.00,0.42
5,NatSound,Publisher,5,94,11.77,0.00,0.39
6,Audio,Publisher,5,94,2.10,0.07,0.10
7,Music,Publisher,5,94,13.16,0.00,0.41
8,Narration,Publisher,5,94,6.63,0.00,0.26
9,Humor,Publisher,5,93,13.85,0.00,0.43


In [49]:
merged_pairwise

,Variable,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,Shots,Publisher,CNN,GoodMorning BadNews,False,True,-2.69,13.00,two-sided,0.02,0.28,bonf,4.443,-1.03
1,Shots,Publisher,CNN,NPR,False,True,-1.03,16.02,two-sided,0.32,1.00,bonf,0.51,-0.33
2,Shots,Publisher,CNN,Tampa Bay Times,False,True,-0.62,26.44,two-sided,0.54,1.00,bonf,0.399,-0.22
3,Shots,Publisher,CNN,The Washington Post,False,True,-1.48,20.02,two-sided,0.15,1.00,bonf,0.762,-0.41
4,Shots,Publisher,CNN,UnderTheDeskNews,False,True,0.44,27.67,two-sided,0.67,1.00,bonf,0.371,0.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,Language,Publisher,NPR,The Washington Post,False,True,1.80,27.76,two-sided,0.08,1.00,bonf,1.102,0.60
191,Language,Publisher,NPR,UnderTheDeskNews,False,True,1.81,27.89,two-sided,0.08,1.00,bonf,1.143,0.61
192,Language,Publisher,Tampa Bay Times,The Washington Post,False,True,0.31,30.17,two-sided,0.76,1.00,bonf,0.333,0.10
193,Language,Publisher,Tampa Bay Times,UnderTheDeskNews,False,True,0.36,30.00,two-sided,0.72,1.00,bonf,0.354,0.13


In [50]:
mean_std_table

,Variable,Value,Mean,Standard Deviation
0,Language,CNN,1.43,0.76
1,Language,GoodMorning BadNews,4.25,0.86
2,Language,NPR,41.76,49.36
3,Language,Tampa Bay Times,20.12,39.14
4,Language,The Washington Post,16.29,34.62
5,Language,UnderTheDeskNews,15.50,32.62
